In [19]:
def find_table_header(json_data, page_idx):
    blocks = json_data["pages"][page_idx]["blocks"]
    
    found_words = {}
    
    for block in blocks:
        for line in block.get("lines", []):
            for word in line.get("words", []):
                value = word["value"]
                found_words[value] = found_words.get(value, []) + [word["geometry"]]
    
    for header in expected_headers:
        matched_count = sum(1 for word in header if word in found_words)
        if matched_count >= len(header) * 0.8: 
            matched_geometries = {word: found_words[word] for word in header if word in found_words}
            return matched_geometries
    
    return None

In [20]:
def find_header_coordinate(matched_geometries, y_tolerance=0.01):
    rows = {}
    
    for word, geometries in matched_geometries.items():
        for geometry in geometries:
            y_coord = geometry[0][1]
            matched_row = None
            for existing_y in rows.keys():
                if abs(existing_y - y_coord) < y_tolerance:
                    matched_row = existing_y
                    break
            
            if matched_row is None:
                rows[y_coord] = []
            else:
                y_coord = matched_row
                
            rows[y_coord].append({
                'word': word,
                'geometry': geometry
            })
    
    header_row = max(rows.items(), key=lambda x: len(x[1]))
    
    header_row_sorted = sorted(header_row[1], key=lambda x: x['geometry'][0][0])
    words_in_order = [item['word'] for item in header_row_sorted]
    geometries = [item['geometry'][0][1] for item in header_row_sorted]
    highest_geometry = max(geometries)
    lowest_geometry = min(geometries)
    
    return {
        'highest_geometry': highest_geometry,
        'lowest_geometry': lowest_geometry,
        'words': words_in_order, #show words for debugging
        'geometries': geometries #show geometries for debugging
    }

In [21]:
def split_json_data(json_data, lowest_geometry, highest_geometry, page_idx):
    above_header = []
    below_header = []

    blocks = json_data["pages"][page_idx]["blocks"]
    
    for block in blocks:
        for line in block.get("lines", []):
            for word in line.get("words", []):
                    word_y_coord = word['geometry'][0][1]
                    word_x_coord = word['geometry'][0][0]
                    
                    if word_y_coord < lowest_geometry:
                        above_header.append({
                            'value': word['value'],
                            'y_coord': word_y_coord,
                            'x_coord': word_x_coord
                        })
                    elif word_y_coord > highest_geometry:
                        below_header.append({
                            'value': word['value'],
                            'confidence': word['confidence'],
                            'geometry': word['geometry'],
                        })

    return above_header, below_header

In [22]:
def get_statement_id(above_header):
    values = {item['value'] for item in above_header}

    with open("database/statement_settings.json") as s:
        statement_settings = json.load(s)


    for setting in statement_settings:
        match_statement_count = sum(1 for value in setting["report_type"] if value in values)
        if match_statement_count == len(setting["report_type"]):
            statement_id = setting["id"]
            lower_limit = setting["lower_limit"]
            return statement_id, lower_limit

In [23]:
def get_table_content(below_header, lower_limit, tolerance=0.05):
    rows = []
    table_content = []
    matched_items = [item for item in below_header if item['value'] in lower_limit]

    if matched_items:
        for mached_item in matched_items:
            item_y = mached_item['geometry'][0][1]
            found_row = False
            for row in rows:
                if abs(row[0]['geometry'][0][1] - item_y) <= tolerance:
                    row.append(mached_item)
                    found_row = True
                    break
            if not found_row:
                rows.append([mached_item])

        max_row = max(rows, key=len)
        limit_coordinate = min(max_row, key=lambda item: item['geometry'][0][1])
        limit_coordinate = limit_coordinate['geometry'][0][1]

        for words in below_header:
            if words["geometry"][0][1] < (limit_coordinate - 0.01):
                table_content.append(words)

        return table_content
    
    else:
        return below_header



In [31]:
import json

file_path = 'sample/scanned/bni-account_statement.json'

expected_headers = [
    {"Posting", "Date", "Effective", "Branch", "Journal", "Transaction", "Description", "Amount", "DB/CR", "Balance"},
    {"Tgl", "Transaksi", "No.", "Dokumen", "Uraian", "Tipe", "Mutasi", "Saldo"},
    {"TIME", "REMARK", "DEBET", "CREDIT", "TELLER", "ID"},
    {"Tanggal", "Keterangan", "Debit", "Kredit", "Saldo", "SEQ"},
    {"Posting", "Date", "Remark", "Reference", "No", "Debit", "Credit", "Balance"},
    {"Date", "Val.Date", "Description", "Reference", "No.", "Debet", "Credit", "Balance"},
    {"Date", "Time", "Value", "Description", "Reference", "No.", "Debit", "Credit", "Saldo"},
    {"Tran", "Date", "Value", "Description", "Debit", "Credit", "Balance"}
]

with open(file_path, 'r') as f:
    json_data = json.load(f)

table_headers, header_coordinates, above_headers, below_headers, table_contents = {}, {}, {}, {}, {}



for page in json_data["pages"]:
    page_idx = page["page_idx"]
    words_without_header = []

    table_header = find_table_header(json_data, page_idx) 
    table_headers[page_idx] = table_header #find table header by page index

    if table_headers[0] is not None and table_header is not None:

        header_coordinate = find_header_coordinate(table_headers[page_idx])
        header_coordinates[page_idx] = header_coordinate #find header coordinate by page index

        above_header, below_header = split_json_data(json_data, header_coordinates[page_idx]['lowest_geometry'], header_coordinates[page_idx]['highest_geometry'], page_idx)
        above_headers[page_idx], below_headers[page_idx] = above_header, below_header
        
        if above_headers[0] != []:
            statement_id, lower_limit = get_statement_id(above_headers[0])
        elif above_header[0] == []:
            print("Account Statement is unknown")
            break

        table_content =  get_table_content(below_headers[page_idx], lower_limit)
        table_contents[page_idx] = table_content

    elif table_headers[0] is not None and table_header is None:
        for block in page.get("blocks", []):
            for line in block.get("lines", []):
                for word in line.get("words", []):
                    words_without_header.append({
                            'value': word['value'],
                            'confidence': word['confidence'],
                            'geometry': word['geometry'],
                        })

        below_headers[page_idx] = words_without_header
        table_content = get_table_content(below_headers[page_idx], lower_limit)
        table_contents[page_idx] = table_content

table_contents
        

{0: [{'value': 'Ledger',
   'confidence': 0.9917080402374268,
   'geometry': [[0.22823424258474578, 0.376953125],
    [0.2770127118644068, 0.3916015625]]},
  {'value': 'Balance:',
   'confidence': 0.999791145324707,
   'geometry': [[0.2770127118644068, 0.3759765625],
    [0.3339209260240113, 0.390625]]},
  {'value': '109,121.00',
   'confidence': 0.9601684212684631,
   'geometry': [[0.9064872439971752, 0.376953125],
    [0.9796549479166667, 0.390625]]},
  {'value': 'TRFIPAY/TOP-UP',
   'confidence': 0.45160433650016785,
   'geometry': [[0.4732879811087571, 0.3935546875],
    [0.5929113700564972, 0.40625]]},
  {'value': 'ECHANNEL',
   'confidence': 0.5891861319541931,
   'geometry': [[0.5940727621822034, 0.3935546875],
    [0.678854387358757, 0.4072265625]]},
  {'value': 'PEMINDAHAN',
   'confidence': 0.9989210367202759,
   'geometry': [[0.4732879811087571, 0.4072265625],
    [0.5708449196680792, 0.419921875]]},
  {'value': 'DARI',
   'confidence': 0.9997615814208984,
   'geometry': [[0